In [1]:
from os import listdir
from os.path import isfile, join

mypath = "/Volumes/TOSHIBA/bandcamp_collection"

onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [5]:
import pandas as pd

onlyfiles_series = pd.Series(onlyfiles, name="Records")

In [15]:
onlyfiles_series

0                           Krush Lumpkin - Rilascio.zip
1                         ._Krush Lumpkin - Rilascio.zip
2      Raccordo Fluviale Subalpino - Raccordo Fluvial...
3      ._Raccordo Fluviale Subalpino - Raccordo Fluvi...
4                 Dori Sorride - A Catturare I Raggi.zip
                             ...                        
811             ._Wedding Acid Group - Easid Bits EP.zip
812                    Unknown Artists - Gomma Piuma.zip
813                  ._Unknown Artists - Gomma Piuma.zip
814         Sprawl + D'Arcangelo Remix - Time Tunnel.zip
815       ._Sprawl + D'Arcangelo Remix - Time Tunnel.zip
Name: Records, Length: 816, dtype: object

In [25]:
onlyfiles_series = pd.Series([x for x in onlyfiles_series if not x.startswith(".")])
onlyfiles_series

0                           Krush Lumpkin - Rilascio.zip
1      Raccordo Fluviale Subalpino - Raccordo Fluvial...
2                 Dori Sorride - A Catturare I Raggi.zip
3                  Kinlaw & Franco Franco - WAVE LED.zip
4                   Reptilian Expo - xperiametaphone.zip
                             ...                        
403                              Sondáil - Polydial.zip
404                     Chris Moss Acid - Lill Chill.zip
405               Wedding Acid Group - Easid Bits EP.zip
406                    Unknown Artists - Gomma Piuma.zip
407         Sprawl + D'Arcangelo Remix - Time Tunnel.zip
Length: 408, dtype: object

In [54]:
df = pd.DataFrame({"Artist": onlyfiles_series.map(lambda x: (x.split("-")[0])), "Album": onlyfiles_series.map(lambda x: (x.split("-")[-1][:-4].strip()))})

In [56]:
df

,Artist,Album
0,Krush Lumpkin,Rilascio
1,Raccordo Fluviale Subalpino,Raccordo Fluviale Subalpino
2,Dori Sorride,A Catturare I Raggi
3,Kinlaw & Franco Franco,WAVE LED
4,Reptilian Expo,xperiametaphone
...,...,...
403,Sondáil,Polydial
404,Chris Moss Acid,Lill Chill
405,Wedding Acid Group,Easid Bits EP
406,Unknown Artists,Gomma Piuma


In [57]:
#df.to_csv("digital_collection.csv")

In [59]:
df.sort_values("Artist")

,Artist,Album
105,01.zip,01
5,AIR Krew,A Walk Through H
26,Abu Ama,AM002
332,Addison Groove,FOOTCRAB VIP REMASTER
44,Adriano Cava,Transient
...,...,...
143,wzrdryAV,Midnight Visions
382,xxoy,Wonderland
370,xxoy.,Vol. 2
366,xxoy.,Vol. 3 by xxoy


In [60]:
df.to_csv("digital_collection.csv")

In [14]:
import glob
import os

root_dir = "/Users/aniel/Rekordbox_database/"
# root_dir needs a trailing slash (i.e. /root/dir/)
for filename in glob.iglob(root_dir + '**/*.mp3', recursive=True):
    print(os.path.basename(filename))
    break

Mina, Bryte, King Doudou - Mina ft. Bryte - Sentah (King Doudou Baile Funk Mix).mp3


In [15]:
tracks = pd.Series([os.path.basename(filename) for filename in glob.iglob(root_dir + '**/*.mp3', recursive=True)])

In [16]:
tracks

0       Mina, Bryte, King Doudou - Mina ft. Bryte - Se...
1                             Motion Ward - Perpetual.mp3
2           Cassius Select - IF YOU DO A LITTLE DANCE.mp3
3                               Pugilist - Intentions.mp3
4                                Loefah - Horror Show.mp3
                              ...                        
1911    EGOLESS - Solidarity in Dub - 04 Cool Down the...
1912    EGOLESS - Solidarity in Dub - 07 Non-Immigrant...
1913    EGOLESS - Solidarity in Dub - 03 Cool Down the...
1914    Pugilist - Pugilist & Another Channel - Fight ...
1915    Pugilist - Pugilist & Another Channel - Fight ...
Length: 1916, dtype: object

In [17]:
import pandas as pd

df_tracks = pd.DataFrame({"Artist": tracks.map(lambda x: (x.split("-")[0])), "Album": tracks.map(lambda x: (x.split("-")[-1][:-4].strip()))})

In [18]:
df_tracks.head()

,Artist,Album
0,"Mina, Bryte, King Doudou",Sentah (King Doudou Baile Funk Mix)
1,Motion Ward,Perpetual
2,Cassius Select,IF YOU DO A LITTLE DANCE
3,Pugilist,Intentions
4,Loefah,Horror Show


In [13]:
df_tracks.to_csv("tracks.csv")

In [19]:
df_tracks.sample(25)

,Artist,Album
855,Piezo,04 Rash
1024,Eben Bull,08 Geology (Digital only)
630,Walton,02 Taiko
1569,Bokeh Allstars,My Perception
1907,EGOLESS,06 Yërmënde (Dub Mix)
586,John T. Gast,05 MANC TK 2
963,Memotone,Off Earth
885,NKC,03 BTD150
136,Mala,02 New Life (Baby Paris)
1356,Huerco S.,03 Plonk III
